In [1]:
from datatalker.ogdp import OGDProxy

ogd = OGDProxy(api_key="579b464db66ec23bdd000001edd87d62b40343d54d7f4653d5d391a7")

/data102/vijay/datatalker/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
catalogs = ogd.get_all(ogd.catalogs)

In [ ]:
# sudo docker volume create --driver local --opt device=/data102/.docker/volumes --opt type=none  mongodb_data
# docker run --name mongodb -d -p 27017:27017 -v mongodb_data:/data/db mongodb/mongodb-community-server

In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")

In [2]:
dt_db = client["datatalker"]
ogd_catalogs = dt_db["ogd_catalogs"]

In [ ]:
ogd_catalogs.insert_many([
    c
    for data in catalogs
    for c in data["data"]["rows"]
])

In [4]:
unique_keys = set()
for doc in ogd_catalogs.find():
    unique_keys.update(doc.keys())

In [5]:
unique_keys

{'_freq_wise_not_updated',
 '_freq_wise_schedule',
 '_freq_wise_total_resources',
 '_freq_wise_updated',
 '_id',
 'api_request_count',
 'body:value',
 'changed',
 'created',
 'data_source',
 'data_time_period_from',
 'data_time_period_to',
 'field_asset_jurisdiction:name',
 'field_ds_govt_type',
 'field_group_name:name',
 'field_ministry_department:name',
 'field_search_keywords',
 'field_sector:name',
 'field_state_department:name',
 'frequency',
 'frequency_data_series',
 'from_api',
 'govt_type',
 'having_api',
 'high_value_dataset',
 'is_api_available',
 'is_priced',
 'is_webservice',
 'keywords',
 'nid',
 'node_alias',
 'ogdp_custom_field',
 'ogdp_view_count',
 'ogpl_module_domain_access',
 'ogpl_module_domain_name',
 'published_date',
 'search_api_datasource',
 'search_api_id',
 'search_api_language',
 'title',
 'uuid',
 'vid',
 'view_count'}

In [6]:
ogd_catalogs.count_documents({})

12451

In [4]:
resources = ogd.get_all(ogd.resources)

In [8]:
rsrc_docs = [
    record
    for resp in resources
    for record in resp["records"]
]

In [10]:
len(rsrc_docs)

282122

In [3]:
ogd_resources = dt_db["ogd_resources"]

In [13]:
ogd_resources.drop()

In [ ]:
failures = list()
for doc in rsrc_docs:
  try:
    ogd_resources.insert_one(doc)
  except:
    failures.append(doc)

In [ ]:
len(failures)

In [4]:
ogd_resources.count_documents({})

282118

In [11]:
ogd_resources.insert_many(rsrc_docs)

OperationFailure: BSONObj size: 34892398 (0x2146A6E) is invalid. Size must be between 0 and 16793600(16MB) First element: _id: ObjectId('6813b725abbc9676584d2a43'), full error: {'ok': 0.0, 'errmsg': "BSONObj size: 34892398 (0x2146A6E) is invalid. Size must be between 0 and 16793600(16MB) First element: _id: ObjectId('6813b725abbc9676584d2a43')", 'code': 10334, 'codeName': 'BSONObjectTooLarge'}

In [ ]:
catalog_uuids = ogd_catalogs.distinct("uuid")
len(catalog_uuids)

In [ ]:
from multiprocessing import cpu_count, Pool
import requests

def get_data(uuid: str, _):
  return requests.get(
    f"https://api.data.gov.in/catalog/{uuid}",
    params={
      "format": "json",
      "offset": 0,
      "limit": 1,
      "api-key": "579b464db66ec23bdd000001edd87d62b40343d54d7f4653d5d391a7"
    }
  )

with Pool(2) as pool:
  args = [
    (uuid, True)
    for uuid in catalog_uuids
  ]
  responses = pool.starmap(get_data, args)

In [ ]:
import os
from urllib.parse import urlparse

os.path.basename(urlparse(responses[0].request.url).path)

In [ ]:
catalog_metadatas = {
  os.path.basename(urlparse(responses[0].request.url).path): r.json()
  for r in responses
}

In [ ]:
ogd_catalogs_meta = dt_db["ogd_catalogs_meta"]

In [ ]:
for uuid, metadata in catalog_metadatas.items():
    metadata["catalog_uuid"] = uuid
    ogd_catalogs_meta.insert_one(metadata)

In [ ]:
ogd_catalogs_meta.drop()

In [ ]:
ogd_catalogs_meta.distinct("message")

In [ ]:
for a in ogd_catalogs_meta.find({"status": { "$ne": "error"}}):
  print(a)

In [ ]:
from IPython.display import HTML
HTML(failed[-1].text)